# Translation notebook

This is the notebook for translation

(More descriptions)

### 1. Import packages

In [1]:
import os
os.getcwd()
import random
import numpy as np

import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TestTubeLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from src.translation_lightning_model import LmForTranslation

### 2. Define parameters

In [2]:
args ={
    'train_data': './my_datasets/IWSLT_2014_TEDtalks/es-en/train_pr',  # Path to training data
    'validation_data': './my_datasets/IWSLT_2014_TEDtalks/es-en/dev2010',  # Path to validation data
    'test_data': './my_datasets/IWSLT_2014_TEDtalks/es-en/test_joined',  # Path to test data
    'src': 'en',  # Source language prefix
    'tgt': 'es',  # Target language prefix
    'max_src_len': 170,  # Maximum number of tokens in the source sentence
    'max_tgt_len': 170,  # Maximum number of tokens in the target sentence
    'save_dir': '../models/iwslt_2014/es-en/es_en_translation_bart_en_jupyter',  # Path to save the model and logs
    'tokenizer': '../pretrained_lms/facebook-bart-base',  # Pretrained tokenizer
    'model': '../pretrained_lms/facebook-bart-base',  # Pretrained model
    'add_adapter': False,  # Include adapter training
    'reduction_factor': 1,  # Adapter's reduction factor (>= 1)
    'label_smoothing': 0.1, # Label smoothing 
    'epochs': 1,  # Number of epochs during training
    'batch_size': 16,  # Batch size
    'grad_accum': 1,  # Gradient accumulation
    'lr': 0.00003,  # Training learning rate
    'warmup': 500,  # Number of warmup steps
    'weight_decay': 0.00003,  # Adam weight decay
    'gpus': 1,  # Number of gpus. 0 for CPU
    'precision': 32,  # Double precision (64), full precision (32) 
                      # or half precision (16). Can be used on CPU, GPU or TPUs.
}

### 3. Initialize Lightning module

In [3]:
# Initialize with a seed
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    
# dataset size. Needed to compute number of steps for the lr scheduler
args['dataset_size'] = sum(1 for line in open(args['train_data'] + '.' + args['src']))

# Define PyTorch Lightning model
model = LmForTranslation(args)
# Include datasets
model.hf_datasets = {'train': args['train_data'],
                     'validation': args['validation_data'],
                     'test': args['test_data']}
print(model.hf_datasets)

# Define logger
logger = TestTubeLogger(
    save_dir=args['save_dir'],
    name='training',
    version=0  # always use version=0
)

# Define checkpoint saver
checkpoint_callback = ModelCheckpoint(
    dirpath=os.path.join(args['save_dir'], "training", "checkpoints"),  # Dir path
    filename='check-{epoch:02d}-{BLEU:.2f}',  # Filename
    save_top_k=1,  # Maximum number of checkpoints to be saved
    verbose=True,  # Verbose
    monitor='BLEU',  # Checkpointing measurement (BLEU validation)
    mode='max',      # Maximize measurement over the validation
    period=1         # Save every epoch
)

print(args)


# Define lightning trainer
trainer = pl.Trainer(gpus=args['gpus'], distributed_backend='dp' if torch.cuda.is_available() else None,
                     track_grad_norm=-1,
                     max_epochs=args['epochs'],
                     max_steps=None,
                     replace_sampler_ddp=False,
                     accumulate_grad_batches=args['grad_accum'],
                     gradient_clip_val=1.0,
                     val_check_interval=1.0,
                     num_sanity_val_steps=2,
                     check_val_every_n_epoch=1,
                     logger=logger,
                     callbacks=checkpoint_callback,
                     progress_bar_refresh_rate=10,
                     precision=args['precision'],
                     amp_backend='native', amp_level='O2',
                     )

/data/injaureg/Desktop/py36_alta/lib64/python3.6/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:488: LightningDeprecationWarning: Argument `period` in `ModelCheckpoint` is deprecated in v1.3 and will be removed in v1.5. Please use `every_n_epochs` instead.
  "Argument `period` in `ModelCheckpoint` is deprecated in v1.3 and will be removed in v1.5."
/data/injaureg/Desktop/py36_alta/lib64/python3.6/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:111: LightningDeprecationWarning: `Trainer(distributed_backend=dp)` has been deprecated and will be removed in v1.5. Use `Trainer(accelerator=dp)` instead.
  f"`Trainer(distributed_backend={distributed_backend})` has been deprecated and will be removed in v1.5."
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


{'train': './my_datasets/IWSLT_2014_TEDtalks/es-en/train_pr', 'validation': './my_datasets/IWSLT_2014_TEDtalks/es-en/dev2010', 'test': './my_datasets/IWSLT_2014_TEDtalks/es-en/test_joined'}
{'train_data': './my_datasets/IWSLT_2014_TEDtalks/es-en/train_pr', 'validation_data': './my_datasets/IWSLT_2014_TEDtalks/es-en/dev2010', 'test_data': './my_datasets/IWSLT_2014_TEDtalks/es-en/test_joined', 'src': 'en', 'tgt': 'es', 'max_src_len': 170, 'max_tgt_len': 170, 'save_dir': '../models/iwslt_2014/es-en/es_en_translation_bart_en_jupyter', 'tokenizer': '../pretrained_lms/facebook-bart-base', 'model': '../pretrained_lms/facebook-bart-base', 'add_adapter': False, 'reduction_factor': 1, 'label_smoothing': 0.1, 'epochs': 1, 'batch_size': 16, 'grad_accum': 1, 'lr': 3e-05, 'warmup': 500, 'weight_decay': 3e-05, 'gpus': 1, 'precision': 32, 'from_pretrained': None, 'dataset_size': 180850}


#### 4. Train model

In [4]:
# Train model
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 139 M 
-------------------------------------------------------
139 M     Trainable params
0         Non-trainable params
139 M     Total params
557.682   Total estimated model params size (MB)


{'train_data': './my_datasets/IWSLT_2014_TEDtalks/es-en/train_pr', 'validation_data': './my_datasets/IWSLT_2014_TEDtalks/es-en/dev2010', 'test_data': './my_datasets/IWSLT_2014_TEDtalks/es-en/test_joined', 'src': 'en', 'tgt': 'es', 'max_src_len': 170, 'max_tgt_len': 170, 'save_dir': '../models/iwslt_2014/es-en/es_en_translation_bart_en_jupyter', 'tokenizer': '../pretrained_lms/facebook-bart-base', 'model': '../pretrained_lms/facebook-bart-base', 'add_adapter': False, 'reduction_factor': 1, 'label_smoothing': 0.1, 'epochs': 1, 'batch_size': 16, 'grad_accum': 1, 'lr': 3e-05, 'warmup': 500, 'weight_decay': 3e-05, 'gpus': 1, 'precision': 32, 'from_pretrained': None, 'dataset_size': 180850}


Validation sanity check: 0it [00:00, ?it/s]

/data/injaureg/Desktop/py36_alta/lib64/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 26 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
Token indices sequence length is longer than the specified maximum sequence length for this model (1028 > 1024). Running this sequence through the model will result in indexing errors
/data/injaureg/Desktop/py36_alta/lib64/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 26 which is the number of cpus on this machine) in the `DataLoader` init to improv

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 0, global step 11280: BLEU reached 30.43455 (best 30.43455), saving model to "/data/injaureg/Desktop/CMCRC/ALTA/ALTA2021_tutorial/models/iwslt_2014/es-en/es_en_translation_bart_en_jupyter/training/checkpoints/check-epoch=00-BLEU=30.43.ckpt" as top 1


### 5. Test model

In [5]:
# Test model
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/data/injaureg/Desktop/py36_alta/lib64/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 26 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Testing: 0it [00:00, ?it/s]

{'avg_val_loss': tensor(2.5985, device='cuda:0'), 'avg_accuracy': tensor(0.3291, device='cuda:0'), 'log': {'vloss': tensor(2.5985, device='cuda:0'), 'vaccuracy': tensor(0.3291, device='cuda:0')}, 'progress_bar': {'vloss': tensor(2.5985, device='cuda:0'), 'vaccuracy': tensor(0.3291, device='cuda:0')}}
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'BLEU': 29.53417205810547}
--------------------------------------------------------------------------------


[{'BLEU': 29.53417205810547}]

### 6. Inference

In [2]:
# Define PyTorch Lightning model
model = LmForTranslation.load_from_checkpoint('../models/iwslt_2014/es-en/es_en_translation_bart_en_jupyter/'
                                              'training/checkpoints/check-epoch=00-BLEU=30.43.ckpt')

sentence = 'Hello my name is Inigo.'
translation = model.translate_example(sentence)
translation

['Hola, mi nombre es Inigo.']